<a href="https://colab.research.google.com/github/HCHegarty/ClassifyingPropositionTypes/blob/Train-test-data/PropositionClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
import warnings
from google.colab import files
warnings.filterwarnings('ignore')

In [3]:
#For logistic regression
from sklearn.linear_model import LogisticRegression

In [26]:
#For naive bayes
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import CategoricalNB

In [5]:
#for linear svc
from sklearn import svm

In [6]:
trainingDataset = files.upload()

Saving FormattedTrainingDataset.csv to FormattedTrainingDataset (1).csv


In [47]:
testingDataset = files.upload()

Saving test.csv to test.csv


In [48]:
df1 = pd.read_csv("test.csv", delimiter=',', header=None)

In [49]:
batch_2 = df1[:1000]
batch_2[1].value_counts()

value     496
policy    153
fact      132
Name: 1, dtype: int64

In [7]:
df = pd.read_csv("FormattedTrainingDataset.csv", delimiter=',', header=None)

In [8]:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

In [9]:
batch_1 = df[:3000]

In [10]:
batch_1[1].value_counts()

value      1682
policy      661
fact        653
,value"       3
,fact"        1
Name: 1, dtype: int64

In [11]:
tokenized = batch_1[0].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

In [12]:
max_len = 0
for i in tokenized.values:
  if len(i) > max_len:
    max_len = len(i)
padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [13]:
np.array(padded).shape

(3000, 116)

In [14]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape


(3000, 116)

In [15]:
input_ids = torch.tensor(padded)
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
  last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [16]:
features = last_hidden_states[0][:,0,:].numpy()

In [17]:
labels = batch_1[1]

In [18]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

In [37]:
#linearSVC score
parameters = {'C': np.linspace(0.0001, 100, 20)}
grid_search = GridSearchCV(svm.LinearSVC(), parameters)
grid_search.fit(train_features, train_labels)

print('best parameters: ', grid_search.best_params_)
print('best scrores: ', grid_search.best_score_)



best parameters:  {'C': 10.526405263157894}
best scrores:  0.7444444444444444


In [41]:
svc_clf = svm.LinearSVC(C=10.526405263157894)
svc_clf.fit(train_features, train_labels)

svc_predicted = svc_clf.predict(test_features)

In [40]:
#Logistic regression score
parameters = {'C': np.linspace(0.0001, 100, 20)}
grid_search = GridSearchCV(LogisticRegression(), parameters)
grid_search.fit(train_features, train_labels)

print('best parameters: ', grid_search.best_params_)
print('best scrores: ', grid_search.best_score_)

best parameters:  {'C': 84.21054210526316}
best scrores:  0.7733333333333332


In [42]:
lr_clf = LogisticRegression(C=84.21054210526316)
lr_clf.fit(train_features, train_labels)


lr_predicted = lr_clf.predict(test_features)

In [43]:
nb_clf = GaussianNB()
nb_clf.fit(train_features, train_labels)
nb_predicted = nb_clf.predict(test_features)
nb_clf.score(test_features, test_labels)

0.636

In [45]:
from sklearn.metrics import classification_report

print("Naive Bayes Score:")
print(classification_report(test_labels, nb_predicted))
print("Linear regression Score:")
print(classification_report(test_labels, lr_predicted))
print("LinearSVC Score:")
print(classification_report(test_labels, svc_predicted))


Naive Bayes Score:
              precision    recall  f1-score   support

        fact       0.43      0.52      0.47       156
      policy       0.61      0.65      0.63       156
       value       0.75      0.67      0.71       438

    accuracy                           0.64       750
   macro avg       0.59      0.61      0.60       750
weighted avg       0.65      0.64      0.64       750

Linear regression Score:
              precision    recall  f1-score   support

        fact       0.60      0.69      0.64       156
      policy       0.86      0.86      0.86       156
       value       0.86      0.82      0.84       438

    accuracy                           0.80       750
   macro avg       0.77      0.79      0.78       750
weighted avg       0.81      0.80      0.80       750

LinearSVC Score:
              precision    recall  f1-score   support

        fact       0.50      0.62      0.56       156
      policy       0.73      0.88      0.80       156
       value  

In [ ]:
from sklearn.dummy import DummyClassifier
clf = DummyClassifier()

scores = cross_val_score(clf, train_features, train_labels)
print("Dummy classifier score: %0.3f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Dummy classifier score: 0.356 (+/- 0.05)
